In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import array, asarray, zeros
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import pickle
import keras
from keras import backend
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.utils import to_categorical
import os

import nltk

#Ignoring the warnings
import warnings
warnings.filterwarnings(action = 'ignore')

/home/sagar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sagar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sagar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sagar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: Futu

In [ ]:
nltk.download('punkt')

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
from sklearn.metrics import roc_auc_score
from keras.layers import LeakyReLU
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from keras.layers import LeakyReLU

In [3]:
def replace_contractions(sentence):
    """Replace contractions in string of text"""
    return contractions.fix(sentence)

  
def words_list(sample):
    words = []
    """Tokenising the corpus"""
    for i in sample: 
        temp = []
        for j in word_tokenize(i):
            temp.append(j.lower())
        temp = normalize(temp)
        words.append(temp)
    return words

  
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

  
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

  
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    """This is the main function which takes all other functions to pre-process the data given"""
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    lemmatize_words(words)
    return words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

  
def lemmatize_words(words):
    """Lemmatize the words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas



In [4]:
#Dictionary of Class names and their index as in labels
label_name = {0: 'Acupuncture', 
              1: 'Chiropractic',
              2: 'Diagnostic Lab',
              3: 'Emergency Services',
              4: 'Human Resource Contact',
              5: 'Maternity',
              6: 'MRI-CAT Scan',
              7: 'Pharmacy Benefit Manager',
              8: 'Physician Visit Office Sick',
              9: 'Psychiatric Out-patient'}

In [12]:
from flask import Flask,jsonify,request
app = Flask(__name__)
textcnn = load_model("./models/Text_CNN.h5")
with open("./pickles/Text_CNN.pickle", 'rb') as handle:
    textcnn_pickle = pickle.load(handle)

# textrnn = load_model("./models/Text_RNN.h5")
# with open("./pickles/Text_RNN.pickle", 'rb') as handle:
#     textrnn_pickle = pickle.load(handle)
@app.route("/", methods=["POST"])
def index():
    req_data = request.get_json()
    s = req_data['sentence']
    word_list = words_list(s)
    X_test = []
    for list_of_words in word_list:
        sentence = ' '.join(x for x in list_of_words)
        X_test.append(sentence)
    encoded_word_list1 = textcnn_pickle.texts_to_sequences(X_test)
    encoded_word_list2 = textrnn_pickle.texts_to_sequences(X_test)
#     Xrnn = pad_sequences(encoded_word_list, maxlen=20, padding='pre')
    Xcnn = pad_sequences(encoded_word_list, maxlen=15, padding='pre')
#     predrnn=textrnn.predict(Xrnn)
#     predrnn=np.argmax(predrnn)
    predcnn=textcnn.predict(Xcnn)
    predcnn=np.argmax(predcnn)
    return jsonify({
                   "predcnn":str(label_name[predcnn])
                   })
if __name__ == '__main__':
	app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

In [10]:
textcnn = load_model("./models/Text_CNN.h5")
with open("./pickles/Text_CNN.pickle", 'rb') as handle:
    textcnn_pickle = pickle.load(handle)
s1=['i am pregnant complications']
word_list = words_list(s1)
X_test = []
for list_of_words in word_list:
    sentence = ' '.join(x for x in list_of_words)
    X_test.append(sentence)
encoded_word_list = textcnn_pickle.texts_to_sequences(X_test)
X = pad_sequences(encoded_word_list, maxlen=15, padding='pre')
print(X)
textcnn._make_predict_function()
pred = textcnn.predict(X)
pred = np.argmax(pred)
print(label_name[pred])

    

[[  0   0   0   0   0   0   0   0   0   0   0   0   0  65 379]]
Maternity


In [7]:
X.shape


(1, 15)